In [ ]:
import numpy as np
from scipy import stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt

### Gather Correlation Data

In [ ]:
encoding_models = ['linear', 'nonlinear_sharedhidden']
subjects = ['F', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
rois = ['PostTemp', 'AntTemp', 'AngularG', 'IFG', 'MFG', 'IFGorb', 'pCingulate', 'dmpfc', 'NonROI']
layer, seq_len, use_ridge, roi_only, batch_size = 8, 10, True, True, 32
roi_masks = np.load('../HP_subj_roi_inds.npy', allow_pickle=True)

In [ ]:
roi_folder = 'roi_voxels' if roi_only else 'all_voxels'
ridge_str = '' if use_ridge else '_noridge'
batch_str = 'nobatch' if (batch_size is None) else 'batch{}'.format(batch_size)
for model in encoding_models:
    print("Model: {}".format(model))
    for roi in rois:
        print("ROI: {}".format(roi))
        for sub in subjects:
            experiment_name = 'subject{}_bert_layer{}_len{}_{}{}_{}'.format(sub, layer, seq_len, model, ridge_str, batch_str)
            output_path = '../fmri_preds/{}/{}/final_predictions.npy'.format(roi_folder, experiment_name)
            corrs = np.nanmean(np.load(output_path, allow_pickle=True).item()['corrs_t'], 0) # Average across four folds
            if roi == 'NonROI':
                roi_mask = ~(roi_masks.item()[sub]['all'])
            else:
                roi_mask = roi_masks.item()[sub][roi]
            roi_corrs = corrs[roi_mask]

In [ ]:
# Print Mean Correlations
# for roi in rois:
#     print('{} Corrs:'.format(roi))
#     mean_lin_corr = np.array(mean_stats['linear'][roi]).mean()
#     mean_nonlin_corr = np.array(mean_stats['nonlinear_sharedhidden'][roi]).mean()
#     stddev_lin_corr = np.std(np.array(mean_stats['linear'][roi]))
#     stddev_nonlin_corr = np.std(np.array(mean_stats['nonlinear_sharedhidden'][roi]))
#     print("Lin={}\nNonlin={}".format(mean_stats['linear'][roi], mean_stats['nonlinear_sharedhidden'][roi]))
#     print("Mean: Lin={}; NonLin={}".format(mean_lin_corr, mean_nonlin_corr))
#     print("StdDev: Lin={}; NonLin={}".format(stddev_lin_corr, stddev_nonlin_corr))
#     print('\n')

In [ ]:
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(20,15))
for idx in range(len(rois)):
    roi = rois[idx]
    i = idx//3
    j = idx%3
    axs[i,j].set_title('{} Mean Correlations'.format(roi))
    axs[i,j].bar([a for a in range(len(mean_stats['linear'][roi]))], mean_stats['linear'][roi], align='center', width=0.4)
    axs[i,j].bar([a+0.4 for a in range(len(mean_stats['nonlinear_sharedhidden'][roi]))], mean_stats['nonlinear_sharedhidden'][roi], align='center', width=0.4)
    axs[i,j].set_ylim(0.0, 0.2)
    axs[i,j].set_xticks(range(len(subjects)))
    axs[i,j].set_xticklabels(subjects)
    if i == 0 and j == 0:
        colors = {'linear':'tab:blue', 'nonlinear_sharedhidden':'tab:orange'}         
        labels = list(colors.keys())
        handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
        axs[i,j].legend(handles, labels)
plt.show()

## Perform paired t-tests

In [ ]:
ts, pvals = [], []
for roi in rois:
    linear_corrs = mean_stats['linear'][roi]
    linear_sgd_corrs = mean_stats['nonlinear_sharedhidden'][roi]
    t, pval = stats.ttest_rel(linear_corrs, linear_sgd_corrs)
    ts.append(t)
    pvals.append(pval)
    print(roi, t, pval)

In [ ]:
one_sided_pvals = [pval/2 for pval in pvals]
print("One-Sided P-values:\n", one_sided_pvals)
adjusted_pvals = statsmodels.stats.multitest.multipletests(one_sided_pvals, alpha=0.05, method='fdr_bh')
print("Adjusted P-values:\n", adjusted_pvals)

In [ ]:
# Print paired t-test results
for roi_idx in range(len(rois)):
    print("{}: \n adjusted p-val={}, reject={}".format(rois[roi_idx], adjusted_pvals[1][roi_idx], adjusted_pvals[0][roi_idx]))